 # El Universo de las Métricas de Distancia



 Este notebook es una guía introductoria al concepto de "distancia" y "similitud".



 **Objetivo:** Entender que no existe una única "mejor" distancia. La elección de la métrica depende fundamentalmente del tipo de datos y del problema que se quiere resolver.



 **Estrategia:** Usaremos un **"Ejemplo Unificador de Bienes Raíces"**. Imaginemos que tenemos los datos de 4 propiedades y queremos ver cuál se "parece" más a la "Propiedad del Cliente".

 ## 0. Configuración e Importaciones

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import (
    euclidean, cityblock, chebyshev, cosine,
    hamming, jaccard, dice
)
from sklearn.preprocessing import StandardScaler

# Configuraciones
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)


 ## 1. El Ejemplo Unificador: Búsqueda de Propiedades



 Un cliente busca una propiedad. Tenemos su propiedad ideal y 3 candidatas.



 **Características:**

 * `precio` (en miles de USD)

 * `area_m2` (metros cuadrados)

 * `piscina` (1 si tiene, 0 si no)

 * `garaje` (1 si tiene, 0 si no)

 * `jardin` (1 si tiene, 0 si no)

 * `lat`, `lon` (coordenadas geográficas)

In [28]:
# --- Creación del DataFrame ---

data = {
    'Propiedad': ['Cliente (Ideal)', 'Casa A', 'Casa B', 'Casa C'],
    'precio': [300, 320, 450, 290],
    'area_m2': [150, 160, 200, 145],
    'piscina': [1, 1, 0, 1],
    'garaje': [1, 1, 1, 0],
    'jardin': [1, 0, 1, 1],
    'lat': [4.601, 4.602, 4.700, 4.600],
    'lon': [-74.080, -74.081, -74.100, -74.079]
}

df = pd.DataFrame(data).set_index('Propiedad')

print("Datos de las propiedades:")
print(df)

# --- Separar los vectores para facilitar los cálculos ---
# Vector del Cliente (nuestro punto de referencia)
v_cliente = df.loc['Cliente (Ideal)']

# Vectores de las candidatas
v_a = df.loc['Casa A']
v_b = df.loc['Casa B']
v_c = df.loc['Casa C']


Datos de las propiedades:
                 precio  area_m2  piscina  garaje  jardin    lat     lon
Propiedad                                                               
Cliente (Ideal)     300      150        1       1       1  4.601 -74.080
Casa A              320      160        1       1       0  4.602 -74.081
Casa B              450      200        0       1       1  4.700 -74.100
Casa C              290      145        1       0       1  4.600 -74.079


 ## 2. Distancias Geométricas (Datos Numéricos)



 Se usan para vectores con números continuos (como `precio` y `area_m2`).



 **¡Paso Crítico: Escalar los datos!** No podemos comparar "300" (precio) con "150" (área). La distancia Euclidiana se vería totalmente dominada por el precio. Debemos normalizar los datos primero.

In [29]:
# --- 2.0. Escalado de Datos Numéricos ---
numeric_features = ['precio', 'area_m2']

scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[numeric_features] = scaler.fit_transform(df[numeric_features])

print("Datos Numéricos Escalados (Media 0, Desv. Est. 1):")
print(df_scaled[numeric_features])

# --- Vectores escalados ---
v_cliente_num = df_scaled.loc['Cliente (Ideal)'][numeric_features].values
v_a_num = df_scaled.loc['Casa A'][numeric_features].values
v_b_num = df_scaled.loc['Casa B'][numeric_features].values
v_c_num = df_scaled.loc['Casa C'][numeric_features].values


Datos Numéricos Escalados (Media 0, Desv. Est. 1):
                   precio   area_m2
Propiedad                          
Cliente (Ideal) -0.620920 -0.636146
Casa A          -0.310460 -0.173494
Casa B           1.707531  1.677113
Casa C          -0.776151 -0.867472


 ### 2.1. Distancia Euclidiana (Norma $L_2$)



 **Qué es:** La distancia en línea recta. Es la hipotenusa de un triángulo rectángulo. Es la distancia "natural" que aprendemos en geometría.



 **Fórmula:** $d(p, q) = \sqrt{\sum_{i=1}^{n} (p_i - q_i)^2}$



 **Interpretación:** Es la distancia "a vuelo de pájaro". Es sensible a grandes diferencias en *cualquier* dimensión (valores atípicos).

In [30]:
print("--- 2.1. Distancia Euclidiana (L2) ---")

dist_eucl_a = euclidean(v_cliente_num, v_a_num)
dist_eucl_b = euclidean(v_cliente_num, v_b_num)
dist_eucl_c = euclidean(v_cliente_num, v_c_num)

print(f"Euclidiana (Cliente vs Casa A): {dist_eucl_a:.4f}")
print(f"Euclidiana (Cliente vs Casa B): {dist_eucl_b:.4f}")
print(f"Euclidiana (Cliente vs Casa C): {dist_eucl_c:.4f}")


--- 2.1. Distancia Euclidiana (L2) ---
Euclidiana (Cliente vs Casa A): 0.5572
Euclidiana (Cliente vs Casa B): 3.2822
Euclidiana (Cliente vs Casa C): 0.2786


 ### 2.2. Distancia Manhattan (Norma $L_1$ o "City Block")



 **Qué es:** La suma de las diferencias absolutas de las coordenadas. Imagina moverte por una ciudad con calles en cuadrícula (como Manhattan); no puedes "cruzar" en diagonal.



 **Fórmula:** $d(p, q) = \sum_{i=1}^{n} |p_i - q_i|$



 **Interpretación:** Es más robusta a valores atípicos que la Euclidiana. Es útil cuando las dimensiones son independientes o el movimiento "diagonal" no tiene sentido.

In [31]:
print("--- 2.2. Distancia Manhattan (L1) ---")

dist_manh_a = cityblock(v_cliente_num, v_a_num)
dist_manh_b = cityblock(v_cliente_num, v_b_num)
dist_manh_c = cityblock(v_cliente_num, v_c_num)

print(f"Manhattan (Cliente vs Casa A): {dist_manh_a:.4f}")
print(f"Manhattan (Cliente vs Casa B): {dist_manh_b:.4f}")
print(f"Manhattan (Cliente vs Casa C): {dist_manh_c:.4f}")


--- 2.2. Distancia Manhattan (L1) ---
Manhattan (Cliente vs Casa A): 0.7731
Manhattan (Cliente vs Casa B): 4.6417
Manhattan (Cliente vs Casa C): 0.3866


 ### 2.3. Distancia de Chebyshev (Norma $L_\infty$ o "Distancia del Tablero de Ajedrez")



 **Qué es:** La máxima diferencia encontrada en *cualquier* dimensión.



 **Fórmula:** $d(p, q) = \max_{i} (|p_i - q_i|)$



 **Interpretación:** Se define por la "peor" característica. Si dos casas son idénticas excepto por el precio, que difiere en 100k, la distancia Chebyshev será 100k (escalado). Útil en logística, donde el tiempo de un proceso está limitado por el componente más lento.

In [32]:
print("--- 2.3. Distancia de Chebyshev (L-infinito) ---")

dist_cheb_a = chebyshev(v_cliente_num, v_a_num)
dist_cheb_b = chebyshev(v_cliente_num, v_b_num)
dist_cheb_c = chebyshev(v_cliente_num, v_c_num)

print(f"Chebyshev (Cliente vs Casa A): {dist_cheb_a:.4f}")
print(f"Chebyshev (Cliente vs Casa B): {dist_cheb_b:.4f}")
print(f"Chebyshev (Cliente vs Casa C): {dist_cheb_c:.4f}")


--- 2.3. Distancia de Chebyshev (L-infinito) ---
Chebyshev (Cliente vs Casa A): 0.4627
Chebyshev (Cliente vs Casa B): 2.3285
Chebyshev (Cliente vs Casa C): 0.2313


 ## 3. Similitud de Orientación (No de Magnitud)



 ### 3.1. Similitud de Coseno



 **Qué es:** Mide el coseno del ángulo entre dos vectores. No mide la *magnitud*, sino la *orientación*.



 **Rango:**

 * `1`: Vectores apuntan exactamente en la misma dirección.

 * `0`: Vectores son ortogonales (no comparten similitud).

 * `-1`: Vectores apuntan en direcciones opuestas.



 **Distancia de Coseno = 1 - Similitud de Coseno**



 **Interpretación:** ¡Extremadamente útil en texto (NLP)! Imagina que el "Documento A" tiene la palabra "fútbol" 2 veces y "Documento B" la tiene 200 veces. Su magnitud es diferente, pero su *tema* (orientación) es similar.



 En nuestro ejemplo, no es muy útil para `precio` y `area`, pero lo calculamos igualmente.

In [33]:
print("--- 3.1. Distancia de Coseno ---")
# Nota: Coseno funciona mejor en datos no centrados (no escalados con StandardScaler),
# así que usamos los datos originales.
v_cliente_orig = df.loc['Cliente (Ideal)'][numeric_features].values
v_a_orig = df.loc['Casa A'][numeric_features].values
v_b_orig = df.loc['Casa B'][numeric_features].values
v_c_orig = df.loc['Casa C'][numeric_features].values

dist_cos_a = cosine(v_cliente_orig, v_a_orig)
dist_cos_b = cosine(v_cliente_orig, v_b_orig)
dist_cos_c = cosine(v_cliente_orig, v_c_orig)

print(f"Dist. Coseno (Cliente vs Casa A): {dist_cos_a:.6f} (Casi idénticos en proporción precio/área)")
print(f"Dist. Coseno (Cliente vs Casa B): {dist_cos_b:.6f}")
print(f"Dist. Coseno (Cliente vs Casa C): {dist_cos_c:.6f}")


--- 3.1. Distancia de Coseno ---
Dist. Coseno (Cliente vs Casa A): 0.000000 (Casi idénticos en proporción precio/área)
Dist. Coseno (Cliente vs Casa B): 0.001031
Dist. Coseno (Cliente vs Casa C): 0.000000


 ## 4. Distancias para Datos Binarios / Sets



 Ahora veremos las características "Sí/No" (Amenidades): `piscina`, `garaje`, `jardin`.



 Estas métricas comparan dos *conjuntos* (Sets).

In [34]:
# --- 4.0. Vectores binarios (Amenidades) ---
binary_features = ['piscina', 'garaje', 'jardin']

v_cliente_bin = df.loc['Cliente (Ideal)'][binary_features].values
v_a_bin = df.loc['Casa A'][binary_features].values
v_b_bin = df.loc['Casa B'][binary_features].values
v_c_bin = df.loc['Casa C'][binary_features].values

print(f"Cliente (Ideal): {v_cliente_bin}")
print(f"Casa A:          {v_a_bin}")
print(f"Casa B:          {v_b_bin}")
print(f"Casa C:          {v_c_bin}")


Cliente (Ideal): [1. 1. 1.]
Casa A:          [1. 1. 0.]
Casa B:          [0. 1. 1.]
Casa C:          [1. 0. 1.]


 ### 4.1. Distancia de Hamming



 **Qué es:** El número de posiciones en las que dos vectores difieren.



 **Interpretación:** ¿Cuántas amenidades tengo que "cambiar" (agregar o quitar) para que una casa sea igual a la otra? Es muy intuitiva.



 * Cliente: `[1, 1, 1]`

 * Casa A: `[1, 1, 0]` -> Difieren en 1 posición. Distancia = 1.

 * Casa C: `[1, 0, 1]` -> Difieren en 1 posición. Distancia = 1.

In [35]:
print("--- 4.1. Distancia de Hamming ---")
# scipy.spatial.distance.hamming devuelve la *proporción* de bits diferentes,
# no el conteo. La multiplicamos por la longitud del vector.
n_features = len(v_cliente_bin)

dist_ham_a = hamming(v_cliente_bin, v_a_bin) * n_features
dist_ham_b = hamming(v_cliente_bin, v_b_bin) * n_features
dist_ham_c = hamming(v_cliente_bin, v_c_bin) * n_features

print(f"Hamming (Cliente vs Casa A): {dist_ham_a:.0f} (Falta 'jardin')")
print(f"Hamming (Cliente vs Casa B): {dist_ham_b:.0f} (Falta 'piscina', sobra 'jardin' - ¡Espera! Hamming solo ve [1,1,1] vs [0,1,1])")
print(f"Hamming (Cliente vs Casa C): {dist_ham_c:.0f} (Falta 'garaje')")


--- 4.1. Distancia de Hamming ---
Hamming (Cliente vs Casa A): 1 (Falta 'jardin')
Hamming (Cliente vs Casa B): 1 (Falta 'piscina', sobra 'jardin' - ¡Espera! Hamming solo ve [1,1,1] vs [0,1,1])
Hamming (Cliente vs Casa C): 1 (Falta 'garaje')


 ### 4.2. Índice (o Distancia) de Jaccard



 **Qué es:** Mide la similitud entre conjuntos.



 **Fórmula (Similitud):** $J(A, B) = \frac{|A \cap B|}{|A \cup B|}$ (Intersección sobre Unión)



 **Distancia Jaccard = 1 - Similitud Jaccard**



 **Interpretación:** De todas las amenidades únicas que existen entre las dos casas, ¿qué porcentaje *comparten*? Es clave que Jaccard **ignora las coincidencias en 0** (es decir, si a ambas casas les falta un "helipuerto", eso no las hace más similares).



 * Cliente: {Piscina, Garaje, Jardin}

 * Casa A: {Piscina, Garaje}

 * Intersección: {Piscina, Garaje} (Tamaño = 2)

 * Unión: {Piscina, Garaje, Jardin} (Tamaño = 3)

 * Similitud Jaccard = 2 / 3 = 0.66

 * Distancia Jaccard = 1 - 0.66 = 0.33

In [36]:
print("--- 4.2. Distancia de Jaccard ---")

dist_jacc_a = jaccard(v_cliente_bin, v_a_bin)
dist_jacc_b = jaccard(v_cliente_bin, v_b_bin)
dist_jacc_c = jaccard(v_cliente_bin, v_c_bin)

print(f"Jaccard (Cliente vs Casa A): {dist_jacc_a:.4f} (1 - 2/3)")
print(f"Jaccard (Cliente vs Casa B): {dist_jacc_b:.4f} (1 - 1/3)")
print(f"Jaccard (Cliente vs Casa C): {dist_jacc_c:.4f} (1 - 2/3)")


--- 4.2. Distancia de Jaccard ---
Jaccard (Cliente vs Casa A): 0.3333 (1 - 2/3)
Jaccard (Cliente vs Casa B): 0.3333 (1 - 1/3)
Jaccard (Cliente vs Casa C): 0.3333 (1 - 2/3)


 ### 4.3. Distancia de Sørensen-Dice



 **Qué es:** Muy similar a Jaccard, pero le da *doble peso* a la intersección (coincidencias positivas).



 **Fórmula (Similitud):** $D(A, B) = \frac{2 \times |A \cap B|}{|A| + |B|}$



 **Distancia Dice = 1 - Similitud Dice**



 **Interpretación:** Es casi intercambiable con Jaccard, pero es más "optimista" sobre las coincidencias.



 * Cliente: {Piscina, Garaje, Jardin} (Tamaño |A| = 3)

 * Casa A: {Piscina, Garaje} (Tamaño |B| = 2)

 * Intersección: 2

 * Similitud Dice = (2 * 2) / (3 + 2) = 4 / 5 = 0.8

 * Distancia Dice = 1 - 0.8 = 0.2

In [37]:
print("--- 4.3. Distancia de Sørensen-Dice ---")

dist_dice_a = dice(v_cliente_bin, v_a_bin)
dist_dice_b = dice(v_cliente_bin, v_b_bin)
dist_dice_c = dice(v_cliente_bin, v_c_bin)

print(f"Dice (Cliente vs Casa A): {dist_dice_a:.4f} (1 - 4/5)")
print(f"Dice (Cliente vs Casa B): {dist_dice_b:.4f} (1 - 2/4)")
print(f"Dice (Cliente vs Casa C): {dist_dice_c:.4f} (1 - 4/5)")


--- 4.3. Distancia de Sørensen-Dice ---
Dice (Cliente vs Casa A): 0.2000 (1 - 4/5)
Dice (Cliente vs Casa B): 0.2000 (1 - 2/4)
Dice (Cliente vs Casa C): 0.2000 (1 - 4/5)


 ## 5. Conclusión: ¿Cuál es la "Mejor" Casa?



 La respuesta depende 100% de la métrica que usemos, porque cada métrica responde a una pregunta de negocio diferente.



 * **Si solo importan `precio` y `area_m2` (Euclidiana):** La **Casa C** es la más cercana (y la A también es muy cercana).

 * **Si solo importan las *amenidades* (Jaccard/Dice):** Las **Casas A y C** están empatadas como las más similares (ambas tienen 1 amenidad diferente).

 * **Si solo importa la *ubicación física* (Haversine):** La **Casa C** es la más cercana por (0.16 Km), seguida de cerca por la Casa A (0.18 Km). La Casa B está muy lejos.



 ### Resumen de Resultados



 (Distancias al Cliente Ideal. **Menor es mejor.**)

In [38]:
resumen_data = {
    'Casa A': [dist_eucl_a, dist_manh_a, dist_cheb_a, dist_cos_a, dist_ham_a, dist_jacc_a, dist_dice_a],
    'Casa B': [dist_eucl_b, dist_manh_b, dist_cheb_b, dist_cos_b, dist_ham_b, dist_jacc_b, dist_dice_b],
    'Casa C': [dist_eucl_c, dist_manh_c, dist_cheb_c, dist_cos_c, dist_ham_c, dist_jacc_c, dist_dice_c],
}
index_metrics = [
    'Num: Euclidiana (Escal.)', 'Num: Manhattan (Escal.)', 'Num: Chebyshev (Escal.)',
    'Num: Coseno (Dist.)', 'Bin: Hamming (Conteo)', 'Bin: Jaccard (Dist.)',
    'Bin: Dice (Dist.)'
]

df_resumen = pd.DataFrame(resumen_data, index=index_metrics)

# Destacar el mínimo en cada fila (la casa "más cercana" para esa métrica)
def highlight_min(s):
    is_min = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_min]

styled_resumen = df_resumen.style.apply(highlight_min, axis=1).format('{:.4f}')

print("--- Resumen de Distancias al 'Cliente (Ideal)' ---")
display(styled_resumen)


--- Resumen de Distancias al 'Cliente (Ideal)' ---


,Casa A,Casa B,Casa C
Num: Euclidiana (Escal.),0.5572,3.2822,0.2786
Num: Manhattan (Escal.),0.7731,4.6417,0.3866
Num: Chebyshev (Escal.),0.4627,2.3285,0.2313
Num: Coseno (Dist.),0.0000,0.0010,0.0000
Bin: Hamming (Conteo),1.0000,1.0000,1.0000
Bin: Jaccard (Dist.),0.3333,0.3333,0.3333
Bin: Dice (Dist.),0.2000,0.2000,0.2000


 **Análisis Final:**



 * **Casa A:** Muy similar en precio/área y ubicación. Le falta 1 amenidad ('jardin').

 * **Casa B:** Muy diferente en precio/área, muy lejos geográficamente y diferente en amenidades. Es la peor candidata en casi todo.

 * **Casa C:** La más similar en precio/área (casi idéntica), la más cercana geográficamente. Le falta 1 amenidad ('garaje').



 **Veredicto:** Si tuviéramos que integrar esto, la **Casa C** es probablemente la mejor opción, seguida muy de cerca por la **Casa A**.